In [83]:
import pandas as pd
import numpy as np
from itertools import chain

from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import HoverTool
import math

In [84]:
df = pd.read_csv("fire.csv")
df.head()

,fire_name,cause,month,year,county,acres,structures,deaths
0,Basin Complex,Lightning,June,2008,Monterey,162818,58,0
1,Big Bar Complex,Lightning,August,1999,Trinity,140948,0,0
2,Camp Fire,Powerlines,November,2018,Butte,153336,18804,85
3,Carr,Human Related,July,2018,"Shasta County,Trinity County",229651,1614,8
4,Cedar,Human Related,October,2003,San Diego,273246,2820,15


In [85]:
dff=df.copy()
dff = (dff.set_index(dff.columns.drop('county',1).tolist())
   .county.str.split(',', expand=True)
   .stack()
   .reset_index()
   .rename(columns={0:'county'})
   .loc[:, dff.columns]
)
dff.drop(['fire_name'],axis=1)
dff.head()

,fire_name,cause,month,year,county,acres,structures,deaths
0,Basin Complex,Lightning,June,2008,Monterey,162818,58,0
1,Big Bar Complex,Lightning,August,1999,Trinity,140948,0,0
2,Camp Fire,Powerlines,November,2018,Butte,153336,18804,85
3,Carr,Human Related,July,2018,Shasta County,229651,1614,8
4,Carr,Human Related,July,2018,Trinity County,229651,1614,8


In [86]:
temp_df = dff.groupby(['year']).sum().reset_index()
temp_df.head()

,year,acres,structures,deaths
0,1932,220000,0,0
1,1970,175425,382,5
2,1977,177866,0,0
3,1987,145980,28,1
4,1999,140948,0,0


In [92]:
#FINAL


output_file("line_chart.html", title="Line Chart")

#p1 : line bar with circle on min()
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
p1 = figure(title="Year-wise total number of acres", y_axis_type="linear", plot_height = 600,
           tools = TOOLS, plot_width = 800,y_range=(110000,860000))
p1.xaxis.axis_label = 'Year'
p1.yaxis.axis_label = 'Total acres'
p1.circle(1999, temp_df.acres.min(), size = 10, color = 'red')
p1.circle(2018, temp_df.acres.max(), size = 10, color = 'red')

p1.line(temp_df.year, temp_df.acres,line_color="purple", line_width = 3)
p1.select_one(HoverTool).tooltips = [
    ('year', '@x'),
    ('acres', '@y'),
]
tab1 = Panel(child=p1, title="Acres affected")


p2 = figure(x_range=df.fire_name, plot_height=600,plot_width=800, title="Number of Deaths")
p2.vbar(x=df.fire_name, top=df.deaths, width=0.9)
p2.xaxis.major_label_orientation = math.pi/3
p2.xgrid.grid_line_color = None
p2.xaxis.axis_label = 'Fire Name'
p2.yaxis.axis_label = 'Deaths'
hover=HoverTool(tooltips=[('Fire Name','@x'),('Deaths','@top')])

p2.add_tools(hover)

tab2 = Panel(child=p2, title="#Deaths")


tabs = Tabs(tabs=[ tab1, tab2 ])
show(tabs)